In [2]:
#**********GET THE DATASET*************
#If the model is already trained you don't need to do this 
from roboflow import Roboflow
rf = Roboflow(api_key="L92yodulFT4LlHfg1I5l")
project = rf.workspace("aiclimbing").project("hold-detector")
dataset = project.version(13).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.226, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Hold-Detector-13 in yolov8:: 100%|██████████| 214/214 [00:00<00:00, 3275.03it/s]


In [3]:
#*********TRAIN THE MODEL**********
#This should already be pretrained so you don't need to do this either
#also it takes like an hour
from ultralytics import YOLO 
import os
# Training.
model = YOLO("yolov8n.pt")
try:
    os.rename("Hold-Detector-13", "datasets")
except:
    pass
try:
    os.rename("datasets/data.yaml", "data.yaml")
except:
    pass
results = model.train(
    data='data.yaml',
    imgsz=1280,
    epochs=50,
    batch=8,
    name='yolov8n_v8_50e',
)

100%|██████████| 6.23M/6.23M [00:00<00:00, 14.1MB/s]


New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.226 🚀 Python-3.10.0 torch-2.2.0.dev20231210 CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, patience=50, batch=8, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v8_50e, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_c

RuntimeError: Dataset 'data.yaml' error ❌ 
Dataset 'data.yaml' images not found ⚠️, missing path '/Users/liam/courses/year4/ai_cs4100/AI-Climbing/datasets/Hold-Detector-13/valid/images'
Note dataset download directory is '/Users/liam/courses/year4/ai_cs4100/AI-Climbing/datasets'. You can update this in '/Users/liam/Library/Application Support/Ultralytics/settings.yaml'

In [4]:
from PIL import Image
import numpy as np
from ultralytics import YOLO
from ultralytics.engine.results import Boxes
import cv2
# Load a pretrained YOLOv8n model
model = YOLO('../runs/detect/yolov8n_v8_50e22/weights/best.pt')

# Gets the image at the specified path as a numpy array
def getImage(path):
    img = cv2.imread(path)
    return img

# Displays the results of model(image)
def dispResults(results):
    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.show()  # show image
        im.save('results.jpg')  # save image

# Removes all bounding boxes in the results of model(image)
# that may be cut off by the sides
def removeEdges(results):
    for r in results:
        boxes_coords = np.array(r.boxes.xyxy)
        boxes_last = np.array(r.boxes.data[:, 4:])
        i=0
        while i < (len(boxes_coords)):
            x1, y1, x2, y2 = boxes_coords[i]
            imh = r.boxes.orig_shape[0]
            imw = r.boxes.orig_shape[1]
            if (x1 < imw / 100.0) or (y1 < imh / 100) or (x2 > imw - imw / 100.0) or (y2 > imh - imh / 100.0):
                boxes_coords = np.delete(boxes_coords, i, 0)
                boxes_last = np.delete(boxes_last, i, 0)
                i -= 1
            i += 1
        new_data = np.concatenate((boxes_coords, boxes_last), axis=1)
        newboxes = Boxes(new_data, r.boxes.orig_shape)
        r.boxes = newboxes
    return results


In [5]:
import os

def getBoxAsImage(results, index):
    r = results[0]
    x1, y1, x2, y2 = np.array(r.boxes.xyxy)[index]
    im_array = np.array(r.orig_img)
    cropped = im_array[round(y1):round(y2), round(x1):round(x2)]
    return cropped
        
def individualHolds(model, img_path, hold_path):
    try:
        os.rmdir(hold_path)
    except:
        pass
    os.mkdir(hold_path)
    for img_name in os.listdir(img_path):   
        img = getImage(f"{img_path}/{img_name}")
        results = model(img)
        results = removeEdges(results)
        r = results[0]
        for i in range(len(r.boxes)):
            im_array = getBoxAsImage(results, i)
            im = Image.fromarray(im_array)
            im.save(f"{hold_path}/{str(i) + '-'}{img_name}")

In [ ]:
def colorDifference (color1, color2):
    c1hsv = cv2.cvtColor(color1, cv2.COLOR_RGB2HSV)
    c2hsv = cv2.cvtColor(color1, cv2.COLOR_RGB2HSV)
    return abs(c1hsv[0] - c2hsv[0])


def getDomColor(img):
    #Use K-means clustering to find the 2 dominant colors
    #https://stackoverflow.com/questions/43111029/how-to-find-the-average-colour-of-an-image-in-python-with-opencv
    
    average = img.mean(axis=0).mean(axis=0)
    pixels = np.float32(img.reshape(-1, 3))
    n_colors = 2
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS

    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    _, counts = np.unique(labels, return_counts=True)
    hsv_palette = []
    for i in range(len(palette)):
        hsv_palette.append(cv2.cvtColor(np.array([[palette[i]]]), cv2.COLOR_BGR2HSV)[0][0])
        hsv_palette[i] = np.array([hsv_palette[i][0], hsv_palette[i][1] * 100, hsv_palette[i][2] * 100 / 255.0])
    #compares the saturations of the two colors (to filter for the wall)
    sats = np.array([col[1] for col in hsv_palette])
    #print(hsv_palette)
    return(hsv_palette[np.argmax(sats)])


In [ ]:
def closeEnough(color2, color1):
    if color1[1] == 0:
        if color2[1] < 20:
            if color2[2] < 50 and color1[2] < 50:
                return True
            elif color2[2] > 50 and color1[2] > 50:
                return True
            else:
                return False
        return False
    else:
        if color2[1] >= color1[1] and color2[2] >= color1[2]:
            diff = abs(color1[0] - color2[0])
            colcheck = min(diff, 360 - diff) < 15
            return colcheck
    return False
def getHoldsNearColor(results, color):
    for r in results:
        boxes_data = np.array(r.boxes.data)
        new_data = np.array([])
        for i in range(len(r.boxes)):
            img = getBoxAsImage(results, i)
            dom = getDomColor(img)
            if closeEnough(dom, color): 
                #print("****")
                print(dom)
                if len(new_data) == 0:
                    new_data = np.array([boxes_data[i]])
                else:
                    new_data = np.append(new_data, [boxes_data[i]], axis=0)
        newboxes = Boxes(new_data, r.boxes.orig_shape)
        r.boxes = newboxes
    return results

In [7]:
im = getImage("../images/coolwall.jpg")
results = (model.predict(im))
#dispResults(results)
# results = model.predict(im)
# greens = getHoldsNearColor(results, np.array([110, 5, 10]))
dispResults(results)


0: 1280x1184 97 holds, 305.0ms
Speed: 20.5ms preprocess, 305.0ms inference, 1.5ms postprocess per image at shape (1, 3, 1280, 1184)


In [ ]:
class Hold:
    def __init__(self, x1, y1, x2, y2, dif):
        self.x1, self.y1, self.x2, self.y2, self.dif = x1, y1, x2, y2, dif

In [ ]:
def getHoldsFromResults(results):
    for r in results:
        holds = []
        boxes_data = np.array(r.boxes.xyxy)
        for box in boxes_data:
            holds.append(Hold(box[0], box[1], box[2], box[3], 0))
    return(holds)